In [4]:
import pandas as pd
from datetime import timezone, timedelta


In [6]:
# Load the data
df = pd.read_csv('Desktop/Daniel_Amuofu/DANIELAMUOFU/source.csv')

In [8]:
df.head()

,Name,Datetime,Amount,Price,Purity
0,ProductA,2022-01-01T01:00:00.000,10,22.09,Impure
1,ProductA,2022-01-01T02:00:00.000,15,24.22,Pure
2,ProductA,2022-01-01T03:00:00.000,10,25.96,Impure
3,ProductA,2022-01-01T04:00:00.000,20,21.16,Impure
4,ProductA,2022-01-01T05:00:00.000,10,20.05,Pure


In [10]:
df.describe()

,Amount,Price
count,24.000000,24.000000
mean,14.791667,22.157917
std,4.772923,1.624612
min,10.000000,20.050000
25%,10.000000,21.220000
50%,15.000000,21.795000
75%,20.000000,22.445000
max,20.000000,26.030000


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Name      24 non-null     object 
 1   Datetime  24 non-null     object 
 2   Amount    24 non-null     int64  
 3   Price     24 non-null     float64
 4   Purity    24 non-null     object 
dtypes: float64(1), int64(1), object(3)
memory usage: 1.1+ KB


In [34]:
# Convert 'datetime' from UTC to UTC+6
df['Datetime'] = pd.to_datetime(df['Datetime'], utc=True)
df['Datetime'] = df['Datetime'].dt.tz_convert('Etc/GMT-6')  # UTC+6

In [32]:
# Create a reference of Product A prices
product_a_prices = df[df['Name'] == 'ProductA'][['Datetime', 'Price']].copy()
product_a_prices.rename(columns={'Price': 'ProductA_Price'}, inplace=True)

In [30]:
# Merge Product A prices with the main dataframe based on matching timestamp
df = df.merge(product_a_prices, on='Datetime', how='left')

In [36]:
df.head()

,Name,Datetime,Amount,Price,Purity,ProductA_Price_x,ProductA_Price_y,ProductA_Price
0,ProductA,2022-01-01 07:00:00+06:00,10,22.09,Impure,22.09,22.09,22.09
1,ProductA,2022-01-01 08:00:00+06:00,15,24.22,Pure,24.22,24.22,24.22
2,ProductA,2022-01-01 09:00:00+06:00,10,25.96,Impure,25.96,25.96,25.96
3,ProductA,2022-01-01 10:00:00+06:00,20,21.16,Impure,21.16,21.16,21.16
4,ProductA,2022-01-01 11:00:00+06:00,10,20.05,Pure,20.05,20.05,20.05


In [38]:
# Define a function to calculate total
def my_total(row):
    price = row['Price']
    
    if row['Purity'] == 'Impure':
        price *= 0.75  # reduce price for impure products by 3/4

    if row['Name'] == 'ProductA':
        return row['Amount'] * price
    elif row['Name'] == 'ProductB':
        if pd.isna(row['ProductA_Price']):
            return None
        base_price = row['ProductA_Price']
        if row['Purity'] == 'Impure':
            base_price *= 0.75
        return row['Amount'] * (price - base_price)
    else:
        return None

In [40]:
df.head()

,Name,Datetime,Amount,Price,Purity,ProductA_Price_x,ProductA_Price_y,ProductA_Price
0,ProductA,2022-01-01 07:00:00+06:00,10,22.09,Impure,22.09,22.09,22.09
1,ProductA,2022-01-01 08:00:00+06:00,15,24.22,Pure,24.22,24.22,24.22
2,ProductA,2022-01-01 09:00:00+06:00,10,25.96,Impure,25.96,25.96,25.96
3,ProductA,2022-01-01 10:00:00+06:00,20,21.16,Impure,21.16,21.16,21.16
4,ProductA,2022-01-01 11:00:00+06:00,10,20.05,Pure,20.05,20.05,20.05


In [46]:
# Apply the function to calculate totals
df['total'] = df.apply(my_total, axis=1)

In [44]:
df.head()

,Name,Datetime,Amount,Price,Purity,ProductA_Price_x,ProductA_Price_y,ProductA_Price,total
0,ProductA,2022-01-01 07:00:00+06:00,10,22.09,Impure,22.09,22.09,22.09,165.675
1,ProductA,2022-01-01 08:00:00+06:00,15,24.22,Pure,24.22,24.22,24.22,363.300
2,ProductA,2022-01-01 09:00:00+06:00,10,25.96,Impure,25.96,25.96,25.96,194.700
3,ProductA,2022-01-01 10:00:00+06:00,20,21.16,Impure,21.16,21.16,21.16,317.400
4,ProductA,2022-01-01 11:00:00+06:00,10,20.05,Pure,20.05,20.05,20.05,200.500


In [222]:
df.head()

,Name,Datetime,Amount,Price,Purity,ProductA_Price,total
0,ProductA,2022-01-01 07:00:00+06:00,10,22.09,Impure,22.09,165.675
1,ProductA,2022-01-01 08:00:00+06:00,15,24.22,Pure,24.22,363.300
2,ProductA,2022-01-01 09:00:00+06:00,10,25.96,Impure,25.96,194.700
3,ProductA,2022-01-01 10:00:00+06:00,20,21.16,Impure,21.16,317.400
4,ProductA,2022-01-01 11:00:00+06:00,10,20.05,Pure,20.05,200.500


In [224]:
# Drop helper column
df.drop(columns=['ProductA_Price'], inplace=True)

In [226]:
df.head()

,Name,Datetime,Amount,Price,Purity,total
0,ProductA,2022-01-01 07:00:00+06:00,10,22.09,Impure,165.675
1,ProductA,2022-01-01 08:00:00+06:00,15,24.22,Pure,363.300
2,ProductA,2022-01-01 09:00:00+06:00,10,25.96,Impure,194.700
3,ProductA,2022-01-01 10:00:00+06:00,20,21.16,Impure,317.400
4,ProductA,2022-01-01 11:00:00+06:00,10,20.05,Pure,200.500


In [230]:
df['total'] = df['total'].round(3)


In [232]:
df.head()

,Name,Datetime,Amount,Price,Purity,total
0,ProductA,2022-01-01 07:00:00+06:00,10,22.09,Impure,165.675
1,ProductA,2022-01-01 08:00:00+06:00,15,24.22,Pure,363.300
2,ProductA,2022-01-01 09:00:00+06:00,10,25.96,Impure,194.700
3,ProductA,2022-01-01 10:00:00+06:00,20,21.16,Impure,317.400
4,ProductA,2022-01-01 11:00:00+06:00,10,20.05,Pure,200.500


In [234]:
# Save the result to result.csv
df.to_csv("result.csv", index=False)